In [1]:
import os
import json
import sys
import logging
import numpy as np
import pandas as pd

/var/folders/4j/wqkxszcs5jl9436sl4hsmprh0000gn/T/ipykernel_58401/3791925645.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
#Loading Dataset
import csv
# Read the 3 different csv files 
books = pd.read_csv('/Users/rabin/College_Project_Seven/Books.csv', low_memory=False)
ratings = pd.read_csv('/Users/rabin/College_Project_Seven/Ratings.csv', encoding='latin-1', delimiter=';')
users = pd.read_csv('/Users/rabin/College_Project_Seven/Users.csv', encoding='latin-1', delimiter=';')


In [9]:

# Manually calculate cosine similarity
def cosine_similarity_manual(matrix):
    """
    Calculate the cosine similarity between rows of a matrix.

    Parameters:
    - matrix: 2D numpy array where each row represents a vector.

    Returns:
    - similarity_scores: 2D numpy array containing cosine similarity scores.
    """
    # Manually calculate dot product
    dot_product = np.array([[np.sum(matrix[i] * matrix[j]) for j in range(matrix.shape[0])] for i in range(matrix.shape[0])])

    # Manually calculate normalization
    norm = np.sqrt(np.sum(matrix**2, axis=1))
    norm_matrix = np.outer(norm, norm)
    
    # Calculate cosine similarity scores
    similarity_scores = dot_product / norm_matrix
    
    return similarity_scores

In [10]:
# Filtered DataFrame based on user ratings
ratings_with_name = ratings.merge(books,on='ISBN')
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] >200
criticalusers = x[x].index

In [11]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(criticalusers)]


In [12]:

# Filtered DataFrame based on famous books
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index


In [13]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]


In [14]:

# Create a pivot table
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')


In [15]:
pt.fillna(0, inplace=True)


In [16]:
similarity_scores_manual = cosine_similarity_manual(pt.values)


In [17]:
similarity_scores_manual = cosine_similarity_manual(pt.values)

# Recommendation function
def recommend(book_name):
    try:
        if book_name in pt.index:
            # index fetch
            index = np.where(pt.index == book_name)[0][0]
            similar_items = sorted(list(enumerate(similarity_scores_manual[index])), key=lambda x: x[1], reverse=True)[1:5]
            
            data = []
            for i in similar_items:
                item = {}
                temp_df = books[books['Book-Title'] == pt.index[i[0]]]
                item['BookTitle'] = temp_df.drop_duplicates('Book-Title')['Book-Title'].values[0]
                #item['ISBN'] = temp_df.drop_duplicates('Book-Title')['ISBN'].values[0]  # Assuming ISBN is a column in your dataset
                #item['ImageURL'] = temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values[0]
                
                data.append(item)
            return data
        else:
            print(f"Book '{book_name}' not found in the dataset.")
            return None
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        return None


